<a href="https://colab.research.google.com/github/anhle/tensorFlow2.x/blob/master/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fashion MNIST
This guide uses the Fashion MNIST dataset which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels). We will train a simple CNN model on the fashion MNIST dataset. 

In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Import TensorFlow
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
import tensorflow_datasets as tfds
from   tensorflow.python.ops import summary_ops_v2
from   tensorflow import keras
from tensorflow.keras import Model, datasets, layers,optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import  time

print(tf.__version__)

2.0.0-alpha0


## Download the fashion mnist dataset
You can access the Fashion MNIST directly from TensorFlow. Import and load the Fashion MNIST data directly from TensorFlow.
Convert the samples from integers to floating-point numbers.

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The labels are an array of integers, ranging from 0 to 9. These correspond to the class of clothing the image represents. Each image is mapped to a single label. Since the class names are not included with the dataset, store them here to use later when plotting the images:

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [25]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y



def mnist_dataset():

  (x, y), (x_val, y_val) = datasets.fashion_mnist.load_data()
  print('x/y shape:', x.shape, y.shape)
  y = tf.one_hot(y, depth=10)
  y_val = tf.one_hot(y_val, depth=10)

  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.shuffle(60000).batch(100)

  ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  ds_val = ds_val.map(prepare_mnist_features_and_labels)
  ds_val = ds_val.shuffle(10000).batch(100)

  sample = next(iter(ds))
  print('sample:', sample[0].shape, sample[1].shape)

  return ds,ds_val

train_ds,val_ds = mnist_dataset()

x/y shape: (60000, 28, 28) (60000,)
sample: (100, 28, 28) (100, 10)


## Explore the data

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Preprocess the data
The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255:

## Build the model

The first layer in this network, tf.keras.layers.Flatten, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two tf.keras.layers.Dense layers. These are densely connected, or fully connected, neural layers. The first Dense layer has 128 nodes (or neurons). The second (and last) layer is a 10-node softmax layer that returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 10 classes.

In [0]:
class MyModel(keras.Model):

    def __init__(self):
        super(MyModel, self).__init__()

        # self.model = keras.Sequential([
        #     layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
        #     layers.Dense(128, activation='relu'),
        #     layers.Dense(128, activation='relu'),
        #     layers.Dense(10)])

        self.layer1 = layers.Dense(128, activation=tf.nn.relu)
        self.layer2 = layers.Dense(128, activation=tf.nn.relu)
        self.layer3 = layers.Dense(10)

    def call(self, x, training=False):

        x = tf.reshape(x, [-1, 28*28])
        x = self.layer1(x)
        x = self.layer2(x)        
        x = self.layer3(x)

        return x
 

model = MyModel()

## Compile the model
Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

* Loss function —This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
* Optimizer —This is how the model is updated based on the data it sees and its loss function.
* Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [0]:
model.compile(optimizer=optimizers.Adam(1e-3),
                  loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

## Train the model
Training the neural network model requires the following steps:

Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
The model learns to associate images and labels.
We ask the model to make predictions about a test set—in this example, the test_images array. We verify that the predictions match the labels from the test_labels array.

In [35]:
model.fit(train_ds.repeat(), epochs=30, steps_per_epoch=500, verbose=1,
              validation_data=val_ds.repeat(),
              validation_steps=2)

Epoch 1/30
500/500 [==============================] - 4s 9ms/step - loss: 0.5472 - accuracy: 0.7387 - val_loss: 0.3431 - val_accuracy: 0.8950
Epoch 2/30
500/500 [==============================] - 4s 8ms/step - loss: 0.3814 - accuracy: 0.8570 - val_loss: 0.3168 - val_accuracy: 0.8900
Epoch 3/30
500/500 [==============================] - 4s 9ms/step - loss: 0.3488 - accuracy: 0.8720 - val_loss: 0.3235 - val_accuracy: 0.9000
Epoch 4/30
500/500 [==============================] - 4s 8ms/step - loss: 0.3218 - accuracy: 0.8805 - val_loss: 0.3036 - val_accuracy: 0.8850
Epoch 5/30
500/500 [==============================] - 4s 7ms/step - loss: 0.3016 - accuracy: 0.8905 - val_loss: 0.3051 - val_accuracy: 0.8950
Epoch 6/30
500/500 [==============================] - 2s 5ms/step - loss: 0.2922 - accuracy: 0.8937 - val_loss: 0.2759 - val_accuracy: 0.8950
Epoch 7/30
500/500 [==============================] - 4s 7ms/step - loss: 0.2741 - accuracy: 0.9011 - val_loss: 0.3243 - val_accuracy: 0.8750
Epoch 

In [41]:
model.fit(train_ds, epochs=30)

Epoch 1/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1108 - accuracy: 0.9578
Epoch 2/30
600/600 [==============================] - 3s 6ms/step - loss: 0.1068 - accuracy: 0.9607
Epoch 3/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1074 - accuracy: 0.9599
Epoch 4/30
600/600 [==============================] - 3s 6ms/step - loss: 0.1042 - accuracy: 0.9608
Epoch 5/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0994 - accuracy: 0.9625
Epoch 6/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0936 - accuracy: 0.9654
Epoch 7/30
600/600 [==============================] - 3s 5ms/step - loss: 0.0949 - accuracy: 0.9637
Epoch 8/30
600/600 [==============================] - 3s 5ms/step - loss: 0.0921 - accuracy: 0.9652
Epoch 9/30
600/600 [==============================] - 3s 6ms/step - loss: 0.0890 - accuracy: 0.9664
Epoch 10/30
600/600 [==============================] - 3s 6ms/step - loss: 0.0902 - accuracy: 0.9651

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.95 (or 95%) on the training data.

## Evaluate accuracy
Next, compare how the model performs on the test dataset:

In [42]:
test_loss, test_acc = model.evaluate(val_ds)

print('\nTest accuracy:', test_acc)

100/100 [==============================] - 1s 7ms/step - loss: 0.7635 - accuracy: 0.8817

Test accuracy: 0.8817


It turns out that the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy represents overfitting. Overfitting is when a machine learning model performs worse on new, previously unseen inputs than on the training data.

## Make predictions
With the model trained, we can use it to make predictions about some images.

In [45]:
predictions = model.predict(val_ds)
predictions[0]
np.argmax(predictions[0])

6